# 프로젝트 설명

In [ ]:
import sys; sys.path.append("..") 

import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
from src.utility import *
project_name = "dual_attention"
createfolder("./data")
createfolder("./model")
createfolder("./result")
createfolder("./graph")

# Data load

In [ ]:
raw_data = loadfile('./data', "raw data", 'csv')
display(raw_data.info())
display(raw_data.describe())

In [ ]:
# Define variables
input_var   = ["FT-3061-2", "FT-3061-3", "FT-3061-4", "FT-3062-1"]
output_var  = ["TT-3061-3", "TT-3061-4", "LT-3061-2"]
process_var = input_var + output_var



# Data preprocessing

## data clustering

In [ ]:
from src import clustering

In [ ]:
kmean_cluster = clustering.KmeanCluster(raw_data, target_var=process_var)
kmean_cluster.test_inertia()

In [ ]:
plot_var = ["FT-3061-2", "FT-3061-3"]+output_var


for i in range(10,15):
    kmean_cluster.model(i)
    kmean_cluster.plot(plot_var=plot_var,
                       fig_ratio=(3,1), subplot=(len(plot_var),1),
                       fig_name=f"cluster{i}")


In [ ]:
kmean_cluster.model(3)

In [ ]:
df_cluster = kmean_cluster.df
df_on = df_cluster[df_cluster['color']=='C2']
df_on.info()

## Domain knowledge

In [ ]:
createfolder('./graph/pairplot')

sns.pairplot(df_on[process_var])
plt.savefig("./graph/pairplot/cluster3 df_on process_var.png", dpi=500)

In [ ]:
# Remove 0 data
path_domain = './data/domain'
name_domain = 'domain dataset'
createfolder(path_domain)

if not exists(path_domain, name_domain):
    df_domain = df_on[(df_on[process_var] > 0).all(axis=1)]
    savefile(df_domain, path_domain, name_domain)
else:
    df_domain = loadfile(path_domain, name_domain)

## Filter continuous data

In [ ]:
from src.dataprocessing import ctsfilter

cts_df_list = ctsfilter(df_domain, 300)

for i, cts_df in enumerate(cts_df_list):
    createfolder('./data/continuous')
    savefile(cts_df, './data/continuous', f"dataset {i+1}")
    #sns.pairplot(cts_df[process_var])
    #plt.savefig(f"./graph/pairplot/continuous {i+1}.png", dpi=500)

# Data preparation

In [ ]:
from src import rnn
import numpy as np

history_length = 60
future_length = 20
step = 1

history
history_var = [, ]
future_var = ["TT-3061-3", "TT-3061-4", "LT-3061-2"]

history_series = []
future_series = []

for i in range(num_data):
    history, future = rnn.data2series(data_list[i], history_length, history_var, future_length, future_var,
                                      step, start_index=0, end_index=None)
    if not i:
        history_series = history
        future_series = future
    else:
        history_series = np.concatenate([history_series, history], axis=0)
        future_series = np.concatenate([future_series, future], axis=0)

In [ ]:
print(f"history series: {history_series.shape}",
      f"future series: {future_series.shape}",
      sep='\n')